In [1]:
Console.WriteLine("Hello, World!")

The below script needs to be able to find the current output cell; this is an easy method to get it.

Hello, World!


In [2]:
public record Person(int Id, string GivenName, string FamilyName,
    DateTime StartDate, int Rating, string FormatString)
{
    public override string ToString()
    {
        if (string.IsNullOrEmpty(FormatString))
            return $"{GivenName} {FamilyName}";
        return string.Format(FormatString, GivenName, FamilyName);
    }
}

public IEnumerable<Person> GetPeople()
{
    var p = new List<Person>()
    {
        new Person(1, "John", "Koenig", new DateTime(1975, 10, 17), 6, ""),
        new Person(2, "Dylan", "Hunt", new DateTime(2000, 10, 2), 8, ""),
        new Person(3, "Leela", "Turanga", new DateTime(1999, 3, 28), 8, "{1} {0}"),
        new Person(4, "John", "Crichton", new DateTime(1999, 3, 19), 7, ""),
        new Person(5, "Dave", "Lister", new DateTime(1988, 2, 15), 9, ""),
        new Person(6, "Laura", "Roslin", new DateTime(2003, 12, 8), 6, ""),
        new Person(7, "John", "Sheridan", new DateTime(1994, 1, 26), 6, ""),
        new Person(8, "Dante", "Montana", new DateTime(2000, 11, 1), 5, ""),
        new Person(9, "Isaac", "Gampu", new DateTime(1977, 9, 10), 4, ""),
    };

    return p;
}

public IEnumerable<int> GetIds()
{
    return GetPeople().Select(p => p.Id);
}

public async Task<Person> GetPerson(int id)
{
    await Task.Delay(1000);
    return GetPeople().SingleOrDefault(p => p.Id == id);
}

In [3]:
var start = DateTime.Now;

foreach (var id in GetIds()) {
    var person = await GetPerson(id);
    Console.WriteLine($"{person.Id}: {person}");
}

var elapsed = DateTime.Now - start;
Console.WriteLine($"Total Time: {elapsed}");

1: John Koenig
2: Dylan Hunt
3: Turanga Leela
4: John Crichton
5: Dave Lister
6: Laura Roslin
7: John Sheridan
8: Dante Montana
9: Isaac Gampu
Total Time: 00:00:09.0773063


In [4]:
var start = DateTime.Now;

var tasks = new List<Task>();

foreach(var id in GetIds())
{
    var task = GetPerson(id);
    tasks.Add(task.ContinueWith(t =>
    {
        var person = t.Result;
        Console.WriteLine($"{person.Id}: {person}");
    }));
}
await Task.WhenAll(tasks);

var elapsed = DateTime.Now - start;
Console.WriteLine($"Total Time: {elapsed}");

5: Dave Lister
1: John Koenig
7: John Sheridan
2: Dylan Hunt
3: Turanga Leela
6: Laura Roslin
8: Dante Montana
9: Isaac Gampu
4: John Crichton
Total Time: 00:00:01.0065492


In [5]:
using System.Threading.Channels;

public async Task Produce(ChannelWriter<Person> writer)
{
    var tasks = new List<Task>();
    foreach(var id in GetIds())
    {
        tasks.Add(PutPersonOnChannel(id, writer));
    }
    await Task.WhenAll(tasks);
}

public async Task PutPersonOnChannel(int id, ChannelWriter<Person> writer)
{
    var person = await GetPerson(id);
    await writer.WriteAsync(person);
}

public async Task Listen(ChannelReader<Person> reader)
{
    await foreach (Person person in reader.ReadAllAsync())
    {
        Console.WriteLine($"{person.Id}: {person}");
    }
}

In [6]:
var start = DateTime.Now;

var channel = Channel.CreateUnbounded<Person>();

var listener = Listen(channel.Reader);
var producer = Produce(channel.Writer);

await producer;
channel.Writer.Complete();
await listener;

var elapsed = DateTime.Now - start;
Console.WriteLine($"Total Time: {elapsed}");

5: Dave Lister
9: Isaac Gampu
8: Dante Montana
6: Laura Roslin
7: John Sheridan
4: John Crichton
3: Turanga Leela
2: Dylan Hunt
1: John Koenig
Total Time: 00:00:01.0249150
